02 — Ingest Grid Operations (Hourly Demand & Generation)
=========================================================
Lumina Forecasting Hub

Pulls hourly data from EIA API v2:
  - electricity/rto/region-data  → actual demand, demand forecast, net generation, interchange
  
For each Balancing Authority in our config, we:
  1. Paginate through the API (5000 rows/page)
  2. Compute forecast error (MW and %)
  3. Load into BigQuery fact_hourly_demand (partitioned by date, clustered by ba_code)

Supports incremental loads — checks the max timestamp already in BQ and only pulls newer data.

Usage in Colab:
  1. Run 01_setup_bigquery_schema.py first
  2. Set your EIA_API_KEY and GCP_PROJECT_ID below
  3. Run all cells

In [ ]:
from google.colab import auth
auth.authenticate_user()

import requests
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta
from google.cloud import bigquery

# ── Config ──────────────────────────────────────────────────────────
EIA_API_KEY    = "YOUR_EIA_API_KEY"       # <-- UPDATE
GCP_PROJECT_ID = "YOUR_GCP_PROJECT_ID"    # <-- UPDATE
BQ_DATASET     = "lumina"
EIA_BASE_URL   = "https://api.eia.gov/v2"

BALANCING_AUTHORITIES = ["PJM", "MISO", "ERCO", "CISO", "ISNE", "NYIS", "SWPP", "SOCO", "TVA", "DUK"]
BACKFILL_START = "2024-01-01T00"          # Hourly format for RTO data

client = bigquery.Client(project=GCP_PROJECT_ID)
print(f"Connected to BigQuery: {GCP_PROJECT_ID}")
print(f"Target BAs: {BALANCING_AUTHORITIES}")

In [ ]:
def fetch_eia_rto(ba_code, data_type, start, end=None, frequency="hourly"):
    """
    Fetch RTO data from EIA API v2 with automatic pagination.
    
    Args:
        ba_code:   Balancing authority code (e.g., 'PJM')
        data_type: 'D' for demand, 'DF' for demand forecast, 'NG' for net gen, 'TI' for interchange
        start:     Start datetime string (e.g., '2024-01-01T00')
        end:       End datetime string (optional)
        frequency: 'hourly' (default) or 'local-hourly'
    
    Returns:
        pd.DataFrame with columns [period, respondent, value, ...]
    """
    route = f"{EIA_BASE_URL}/electricity/rto/region-data/data/"
    
    all_records = []
    offset = 0
    page_size = 5000
    
    while True:
        params = {
            "api_key": EIA_API_KEY,
            "frequency": frequency,
            "data[0]": "value",
            "facets[respondent][]": ba_code,
            "facets[type][]": data_type,
            "sort[0][column]": "period",
            "sort[0][direction]": "asc",
            "offset": offset,
            "length": page_size,
        }
        if start:
            params["start"] = start
        if end:
            params["end"] = end
        
        resp = requests.get(route, params=params)
        resp.raise_for_status()
        body = resp.json()
        
        data = body.get("response", {}).get("data", [])
        total = int(body.get("response", {}).get("total", 0))
        
        if not data:
            break
        
        all_records.extend(data)
        offset += page_size
        
        print(f"    [{ba_code}/{data_type}] fetched {len(all_records)}/{total} rows", end="\r")
        
        if offset >= total:
            break
        
        time.sleep(0.3)  # Rate limiting courtesy
    
    print(f"    [{ba_code}/{data_type}] fetched {len(all_records)}/{total} rows — done")
    return pd.DataFrame(all_records)

In [ ]:
def get_max_timestamp(ba_code):
    """
    Returns the max timestamp_utc for a given BA already in BigQuery.
    Returns BACKFILL_START if table is empty for that BA.
    """
    query = f"""
    SELECT MAX(timestamp_utc) AS max_ts
    FROM `{GCP_PROJECT_ID}.{BQ_DATASET}.fact_hourly_demand`
    WHERE ba_code = '{ba_code}'
    """
    try:
        result = client.query(query).to_dataframe()
        max_ts = result["max_ts"].iloc[0]
        if pd.isna(max_ts):
            return BACKFILL_START
        # Return 1 hour after max to avoid overlap
        next_ts = pd.Timestamp(max_ts) + pd.Timedelta(hours=1)
        return next_ts.strftime("%Y-%m-%dT%H")
    except Exception:
        return BACKFILL_START

In [ ]:
def ingest_ba(ba_code, mode="incremental"):
    """
    Ingest all hourly data for one Balancing Authority.
    
    Fetches 4 series (demand, demand_forecast, net_generation, interchange),
    merges them on timestamp, computes forecast error, and loads to BQ.
    """
    if mode == "incremental":
        start = get_max_timestamp(ba_code)
    else:
        start = BACKFILL_START
    
    end = datetime.utcnow().strftime("%Y-%m-%dT%H")
    print(f"\n{'='*60}")
    print(f"  BA: {ba_code} | Range: {start} → {end} | Mode: {mode}")
    print(f"{'='*60}")
    
    # Fetch each data type
    data_types = {
        "D":  "demand_mw",
        "DF": "demand_forecast_mw",
        "NG": "net_generation_mw",
        "TI": "interchange_mw",
    }
    
    dfs = {}
    for dtype_code, col_name in data_types.items():
        df = fetch_eia_rto(ba_code, dtype_code, start, end)
        if df.empty:
            print(f"    WARNING: No data for {ba_code}/{dtype_code}")
            dfs[col_name] = pd.DataFrame(columns=["period", col_name])
            continue
        df = df.rename(columns={"value": col_name})
        df[col_name] = pd.to_numeric(df[col_name], errors="coerce")
        dfs[col_name] = df[["period", col_name]].copy()
    
    # Merge all series on period
    if all(d.empty for d in dfs.values()):
        print(f"    SKIP: No data returned for {ba_code}")
        return 0
    
    merged = None
    for col_name, df in dfs.items():
        if df.empty:
            continue
        if merged is None:
            merged = df
        else:
            merged = merged.merge(df, on="period", how="outer")
    
    if merged is None or merged.empty:
        print(f"    SKIP: Empty merge for {ba_code}")
        return 0
    
    # Clean up
    merged["ba_code"] = ba_code
    merged["timestamp_utc"] = pd.to_datetime(merged["period"], utc=True)
    
    # Compute forecast error
    if "demand_mw" in merged.columns and "demand_forecast_mw" in merged.columns:
        merged["forecast_error_mw"] = merged["demand_mw"] - merged["demand_forecast_mw"]
        merged["forecast_error_pct"] = np.where(
            merged["demand_forecast_mw"] != 0,
            (merged["forecast_error_mw"] / merged["demand_forecast_mw"]) * 100,
            np.nan
        )
    else:
        merged["forecast_error_mw"] = np.nan
        merged["forecast_error_pct"] = np.nan
    
    # Select final columns
    final_cols = [
        "timestamp_utc", "ba_code",
        "demand_mw", "demand_forecast_mw", "net_generation_mw", "interchange_mw",
        "forecast_error_mw", "forecast_error_pct",
    ]
    for col in final_cols:
        if col not in merged.columns:
            merged[col] = np.nan
    
    result = merged[final_cols].copy()
    result = result.dropna(subset=["timestamp_utc"])
    result = result.sort_values("timestamp_utc").reset_index(drop=True)
    
    # Load to BigQuery
    table_ref = f"{GCP_PROJECT_ID}.{BQ_DATASET}.fact_hourly_demand"
    job_config = bigquery.LoadJobConfig(write_disposition="WRITE_APPEND")
    
    job = client.load_table_from_dataframe(result, table_ref, job_config=job_config)
    job.result()
    
    print(f"    Loaded {len(result)} rows to fact_hourly_demand")
    return len(result)

In [ ]:
MODE = "backfill"  # Change to "incremental" after first run

total_rows = 0
results = {}

for ba in BALANCING_AUTHORITIES:
    try:
        n = ingest_ba(ba, mode=MODE)
        results[ba] = {"status": "success", "rows": n}
        total_rows += n
    except Exception as e:
        results[ba] = {"status": "error", "error": str(e)}
        print(f"    ERROR for {ba}: {e}")

print(f"\n{'='*60}")
print(f"  INGESTION COMPLETE")
print(f"  Total rows loaded: {total_rows:,}")
print(f"{'='*60}")

df_results = pd.DataFrame(results).T
print(df_results)

In [ ]:
quality_query = f"""
WITH stats AS (
    SELECT
        ba_code,
        COUNT(*) AS row_count,
        MIN(timestamp_utc) AS earliest,
        MAX(timestamp_utc) AS latest,
        ROUND(AVG(demand_mw), 1) AS avg_demand_mw,
        ROUND(AVG(ABS(forecast_error_pct)), 2) AS mean_abs_pct_error,
        COUNTIF(demand_mw IS NULL) AS null_demand,
        COUNTIF(demand_forecast_mw IS NULL) AS null_forecast,
    FROM `{GCP_PROJECT_ID}.{BQ_DATASET}.fact_hourly_demand`
    GROUP BY ba_code
)
SELECT * FROM stats ORDER BY row_count DESC
"""

df_quality = client.query(quality_query).to_dataframe()
print("\n=== Data Quality Report: fact_hourly_demand ===")
print(df_quality.to_string(index=False))

In [ ]:
import matplotlib.pyplot as plt

viz_query = f"""
SELECT
    timestamp_utc,
    ba_code,
    demand_mw,
    demand_forecast_mw,
    forecast_error_pct
FROM `{GCP_PROJECT_ID}.{BQ_DATASET}.fact_hourly_demand`
WHERE ba_code = 'PJM'
    AND timestamp_utc >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 7 DAY)
ORDER BY timestamp_utc
"""

df_viz = client.query(viz_query).to_dataframe()

if not df_viz.empty:
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
    
    ax1.plot(df_viz["timestamp_utc"], df_viz["demand_mw"], label="Actual Demand", linewidth=1.2)
    ax1.plot(df_viz["timestamp_utc"], df_viz["demand_forecast_mw"], label="EIA Forecast", linewidth=1.2, linestyle="--")
    ax1.set_ylabel("MW")
    ax1.set_title("PJM — Actual vs. Forecast Demand (Last 7 Days)")
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    ax2.bar(df_viz["timestamp_utc"], df_viz["forecast_error_pct"], width=0.03, alpha=0.7, color="coral")
    ax2.axhline(0, color="black", linewidth=0.5)
    ax2.set_ylabel("Forecast Error %")
    ax2.set_xlabel("Time (UTC)")
    ax2.set_title("Forecast Error Distribution")
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary stats
    mape = df_viz["forecast_error_pct"].abs().mean()
    print(f"\nPJM 7-Day MAPE: {mape:.2f}%")
else:
    print("No data available for visualization yet.")